Azure AI Search with LlamaIndex

This code demonstrates how to use Azure AI Search with Azure OpenAI and the LlamaIndex data framework

Install packages

In [2]:
! pip install -r requirements.txt --quiet

You should consider upgrading via the '/Users/sithukaungset/Azure-AI-Search-prompthon/venv/bin/python3 -m pip install --upgrade pip' command.


Load .env file

In [8]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Make sure your .env file has values for the following environment variables
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
# Llama Index does not support RBAC authentication, an API key is required
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
if len(azure_openai_key) == 0:
    raise Exception("API key required")
azure_openai_embedding_model = os.environ["AZURE_OPENAI_EMBEDDING_MODEL_NAME"]
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
azure_openai_chatgpt_deployment = os.environ["AZURE_OPENAI_CHATGPT_DEPLOYMENT"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]
embedding_dimensions = os.environ["AZURE_OPENAI_EMBEDDING_DIMENSIONS"]

Configure an embeddings instance

In [9]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI

embeddings = AzureOpenAIEmbedding(
    model_name=azure_openai_embedding_model,
    deployment_name=azure_openai_embedding_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

llm = AzureOpenAI(
    deployment_name=azure_openai_chatgpt_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

In [15]:
! pip install --upgrade jupyter ipywidgets docx2txt

  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
  Using cached ipywidgets-8.1.2-py3-none-any.whl (139 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 20.4 MB/s eta 0:00:00a 0:00:01
  Using cached jupyterlab_widgets-3.0.10-py3-none-any.whl (215 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 KB 14.4 MB/s eta 0:00:00
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 KB 7.0 MB/s eta 0:00:00
  Using cached tinycss2-1.2.1-py3-none-any.whl (21 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 KB

In [11]:
! pip install torch transformers python-pptx Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 11.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 19.5 MB/s eta 0:00:0000:0100:01
  Using cached python_pptx-0.6.23-py3-none-any.whl (471 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.4/393.4 KB 7.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 11.2 MB/s eta 0:00:0000:0100:01
  Using cached MarkupSafe-2.1.5-cp310-cp310-macosx_10_9_universal2.whl (18 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
You should consider upgrading via the '/Users/sithukaungset/Azure-AI-Search-prompthon/venv/bin/p

Upload, vectorize, and index documents

This step reads PDFs from a local folder, calls the embedding model for vectorization, and then calls a search client to index the content on Azure AI Search.



In [16]:

from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement, MetadataIndexFieldType
from llama_index.core import StorageContext, VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from azure.search.documents.indexes import SearchIndexClient

metadata_fields = {
    "author": "author",
    "theme": ("topic", MetadataIndexFieldType.STRING),
    "director": "director",
}

vector_store = AzureAISearchVectorStore(  
    search_or_index_client=SearchIndexClient(endpoint=endpoint, credential=credential),  
    filterable_metadata_field_keys=metadata_fields,
    index_name=index_name,  
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,  
    id_field_key="id",  
    chunk_field_key="content",  
    embedding_field_key="content_vector",  
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    embedding_dimensionality=embedding_dimensions,
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
Settings.llm = llm
Settings.embed_model = embeddings
directory = os.path.abspath(os.path.join("..", "data", "documents"))
documents = SimpleDirectoryReader(directory).load_data()
index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context)

Failed to load file /Users/sithukaungset/Azure-AI-Search-prompthon/data/documents/PPT샘플(한글)_세미나발표.pptx with error: cannot find loader for this WMF file. Skipping...


Ignoring wrong pointing object 2 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 60 0 (offset 0)
Ignoring wrong pointing object 65 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 100 0 (offset 0)


HttpResponseError: () The request is invalid. Details: The property 'id' does not exist on type 'search.documentFields'. Make sure to only use property names that are defined by the type.
Code: 
Message: The request is invalid. Details: The property 'id' does not exist on type 'search.documentFields'. Make sure to only use property names that are defined by the type.

Perform a vector similarity search


In [ ]:
query_engine = index.as_query_engine(llm)
response = query_engine.query("What is included in my Northwind Health Plus plan that is not in standard?")
print(response)